In [1]:
import pandas as pd
import os
import pickle
from tqdm import tqdm

Paths

In [2]:
DATA_PATH = '../data'
DATASET_PATH = os.path.join(DATA_PATH, 'ml-100k')

Load genres list

In [3]:
with open(os.path.join(DATASET_PATH, 'u.genre'), 'r') as f:
    data = f.readlines()
    genres_list = [d.strip().split('|')[0].lower().replace("'", "") if d.find("oir") == -1 else "noir" for d in data][1:]

genres_list

['action',
 'adventure',
 'animation',
 'childrens',
 'comedy',
 'crime',
 'documentary',
 'drama',
 'fantasy',
 'noir',
 'horror',
 'musical',
 'mystery',
 'romance',
 'sci-fi',
 'thriller',
 'war',
 'western']

Load dataset

In [4]:
ratings = pd.read_csv(os.path.join(DATASET_PATH, 'u.data'), sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])
movies = pd.read_csv(os.path.join(DATASET_PATH, 'u.item'), sep='|', encoding="latin-1", names=['item_id', 'title', 'release_date', 'video_release_date', 'IMDb_URL', 'unknown', *genres_list])
users = pd.read_csv(os.path.join(DATASET_PATH, 'u.user'), sep='|', names=[' user_id', 'age', 'gender', 'occupation', 'zip_code'])

movies = movies.drop(['release_date', 'title','video_release_date', 'IMDb_URL'], axis=1, inplace=False)
users = users.drop(['zip_code'], axis=1, inplace=False)

dataset = pd.merge(ratings, movies, on='item_id')

# drop row if unknown is 1
dataset = dataset[dataset['unknown'] != 1]
dataset = dataset.drop(['unknown'], axis=1, inplace=False)

dataset

,user_id,item_id,rating,timestamp,action,adventure,animation,childrens,comedy,crime,...,fantasy,noir,horror,musical,mystery,romance,sci-fi,thriller,war,western
0,196,242,3,881250949,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,186,302,3,891717742,0,0,0,0,0,1,...,0,1,0,0,1,0,0,1,0,0
2,22,377,1,878887116,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,244,51,2,880606923,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,1
4,166,346,1,886397596,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,880,476,3,880175444,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
99996,716,204,5,879795543,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
99997,276,1090,1,874795795,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
99998,13,225,2,882399156,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# def load_glove(
#     words=None,
#     glove_path="../glove.6B/glove.6B.300d.txt",
#     save_path="../data/glove.json",
# ):

#     if "glove.json" in os.listdir("../data/"):
#         with open(save_path, "r") as f:
#             glove = json.load(f)
#             f.close()
#     else:
#         glove = {}
#         with open(glove_path, "r") as f:
#             for line in tqdm(f):
#                 values = line.split()
#                 word = values[0]
#                 if words is None or word in words:
#                     vector = np.asarray(values[1:], "float32")
#                     glove[word] = vector.tolist()
#             f.close()

#         with open(save_path, "w") as f:
#             json.dump(glove, f)
#             f.close()

#     return glove

# glove = load_glove(words=genres_list)

# glove = np.array([glove[genre] for genre in genres_list])

sparse_vecs = []
# semantic_vecs = []

for index, row in tqdm(dataset.iterrows()):
    # extract one-hot encoded genres
    sparse_vec = row[4:].to_numpy() # (18,)
    # sparse_vec_reshaped = [np.zeros(300) if ele == 0 else np.ones(300) for ele in sparse_vec]

    # # element-wise multiplication
    # semantic_vec = np.multiply(glove, sparse_vec_reshaped) # (18, 300)

    sparse_vecs.append(sparse_vec)
    # semantic_vecs.append(semantic_vec)

dataset['sparse_vec'] = sparse_vecs
# dataset['semantic_vec'] = semantic_vecs

dataset.drop(genres_list, axis=1, inplace=True)

pickle.dump(dataset, open(os.path.join(DATA_PATH, 'dataset.pkl'), 'wb'))
pickle.dump(users, open(os.path.join(DATA_PATH, 'users.pkl'), 'wb'))

99990it [00:05, 16906.08it/s]
